# Camembert fine-tuning

### Cuda debug

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

## Start fine-tuning

In [ ]:
import pandas as pd
import accelerate
import transformers
print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)


In [ ]:
training_data_pd = pd.read_csv('training_data.csv')
unlabelled_data_pd = pd.read_csv('unlabelled_test_data.csv')
sample_submission_pd = pd.read_csv('sample_submission.csv')

training_data_pd = training_data_pd.drop('id', axis=1)
#unlabelled_data_pd = unlabelled_data_pd.drop('id', axis=1)
#sample_submission_pd = sample_submission_pd.drop('id', axis=1)


#### double the dataframe test

In [ ]:
import pandas as pd

training_data_pd = pd.concat([training_data_pd, training_data_pd], ignore_index=True)

#print(training_data_pd)

In [ ]:
training_data_pd = training_data_pd.sample(frac=1, random_state=42).reset_index(drop=True)

#### augment data test

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import europarl_raw

nltk.download('punkt')
nltk.download('europarl_raw')

def process_text(input_text, difficulty_level):
    french_tokenizer = PunktSentenceTokenizer(europarl_raw.french.raw())

    sentences = french_tokenizer.tokenize(input_text)
    
    df = pd.DataFrame({
        'sentence': sentences,
        'difficulty': [difficulty_level] * len(sentences)
    })
    
    return df

input_text = "Bonjour ! Je m’appelle Claire. À l’école, il y a beaucoup de langues. Je parle anglais et je parle aussi un peu d’italien. Maintenant, j’étudie le français.Je viens d’apprendre comment poser les questions. Comment t’appelles-tu ? Combien de langues est-ce que tu parles ? Je vais visiter la France avec un ami. Il s’appelle Thomas. Nous allons à Paris, pour voir la tour Eiffel et la pyramide du Louvre.Est-ce que tu vas à Paris aussi ? Est-ce que tu aimes le français ? C’est une jolie langue, n’est-ce pas ? Bonjour, je m’appelle Élisabeth. J’ai vingt-neuf ans et je suis française. Je suis née à Lyon mais j’ai grandi à Marseille. Je suis allée à l’université à Paris. Maintenant, j’habite à Strasbourg et je suis ingénieur.Je vous présente ma famille. Mon père est professeur de maths et ma mère est avocate. Ils sont mariés depuis trente ans. J’ai trois frères et quatre sœurs – c’est une grande famille, n’est-ce pas ? Je suis mariée depuis six ans. Mon mari est médecin et nous avons deux enfants : une fille et un fils. J’aime beaucoup voyager : je parle espagnol et allemand. Je viens de rentrer du Portugal, et l’année prochaine, je vais visiter l’Angleterre pour la première fois. Il y a beaucoup d’autres pays que je veux visiter, mais je n’ai pas assez de temps. Bien sûr, je dois travailler, comme tout le monde. Au revoir ! Je vais manger quelque chose … mais quoi ? J’aime la salade, mais je n’ai pas de laitue. Il y a des pâtes, mais il n’y a pas de sauce. J’ai de la soupe, mais il fait trop chaud pour la soupe. Il y a du riz, mais je ne mange pas de riz. Je vois trois pommes de terre, mais elles ne sont pas bonnes. Je vais aller au restaurant ! Bonjour, je m’appelle Élisabeth. J’ai vingt-neuf ans et je suis française. Je suis née à Lyon mais j’ai grandi à Marseille. Je suis allée à l’université à Paris. Maintenant, j’habite à Strasbourg et je suis ingénieur. Je vous présente ma famille. Mon père est professeur de maths et ma mère est avocate. Ils sont mariés depuis trente ans. J’ai trois frères et quatre sœurs – c’est une grande famille, n’est-ce pas ? Je suis mariée depuis six ans. Mon mari est médecin et nous avons deux enfants : une fille et un fils. J’aime beaucoup voyager : je parle espagnol et allemand. Je viens de rentrer du Portugal, et l’année prochaine, je vais visiter l’Angleterre pour la première fois. Il y a beaucoup d’autres pays que je veux visiter, mais je n’ai pas assez de temps. Bien sûr, je dois travailler, comme tout le monde. Au revoir ! "
new_df = process_text(input_text, 'A1')

input_text = "Le premier juillet, c’est la Fête du Canada, la fête nationale du Canada. Dans les années où le premier juillet tombe un dimanche, la plupart des célébrations ont quand même lieu ce jour, mais le lundi 2 juillet sert de fête férié. La fête du Canada commémore le fait de signer l’Acte de l’Amérique du Nord britannique (l’Acte ANB), qui a réuni trois colonies britannique en une dominion fédérale et a défini la plupart du gouvernement canadien. Elle s’appelle à l’origine la Fête du Dominion, mais en 1982, quand le gouvernement britannique a enfin cédé tout contrôle sur le Canada, le premier juillet a été rebaptisé la fête du Canada, et l’Acte ANB a été renommé l’Acte constitutionnelle. La fête du Canada s’appelle aussi l’anniversaire du Canada, et avant 1982, le Fête de la Confédération. Comme les fêtes nationales des USA (le 4 juillet) et de France (le 14 juillet), la Fête du Canada célèbre la création du pays moderne avec des événements de plein air comme des défilés, feu d’artifice et pique-niques. La décoration indispensable d’un Noël français est la crèche, qui se trouve dans les églises, dans les maisons et parfois sur les places publiques. En Provence, les crèches sont souvent un mélange de religion et de quotidien, présentant non seulement la naissance du Christ, mais aussi le village et le mode de vie provençaux, avec des fermes, des magasins et des personnages typiques de la région. Les petites figurines qui peuplent les crèches s’appellent les santons (littéralement, « petits saints »), et la ville d’Aubagne en est la capitale mondiale. Faits en terre cuite ou en argile, on peut acheter des santons peints ou à peindre et créer son propre village provençal. À part les personnages de la bible (Joseph, Marie, l’enfant Jésus, les Rois Mages), la crèche provençale met en vedette beaucoup de personnages typiques parmi lesquels : Il y a aussi beaucoup d’animaux domestiques (âne, bœuf, mouton…), de végétaux (le houx, le laurier-tin, la mousse, l’olivier, le pyracantha, sans oublier le blé de la Sainte Barbe) et aussi, bien sûr, de bâtiments : le moulin, l’étable, les magasins, les mas. Quatre personnages ne sont pas mis dans la crèche au début. En rentrant de la messe de minuit, le benjamin place l’enfant Jésus entre ses Saints Parents. Enfin, le 6 janvier, les Rois mages arrivent et complètent la crèche. Normalement, la crèche est défaite le 2 février avant de fêter la Chandeleur. L’une des questions qu’on me pose le plus souvent, c’est du genre « Je suis en train de perdre mon français ! Comment puis-je le maintenir quand je ne suis pas en France ? » Voici ma réponse. Tout d’abord, je recommande vivement que vous deveniez membre de l’Alliance française. Cette organisation internationale à but non lucratif promeut la langue de Molière partout dans le monde, avec plus de 800 succursales dans environ 135 pays. Elle propose non seulement des cours de français pour tous les niveaux, mais aussi un programme d’activités (conférences, films, dîners…) pour que les adhérents, dont des locuteurs natifs, puissent se réunir pour discuter en français. S’il n’y a pas d’Alliance française près de chez vous, cherchez un club dans une université ou un centre universitaire dans le coin, ou sur le site Meetup, ou bien créez–en un vous-même ! Pourquoi pas ? Vous avez envie de parler français, il y a sans doute d’autres gens dans votre ville qui le veulent aussi. Surtout, ne soyez pas craintif ! La timidité est l’ennemi de l’apprentissage et, surtout, de la pratique d’une langue étrangère. Personne ne va se moquer de vous, vous êtes tous là pour vous éduquer et vous divertir, donc profitez de l’occasion pour parler, avec tous vos défauts. Les réunions hebdomadaires ou mensuelles sont utiles, mais il est également important de faire un peu de français tous les jours : si vous n’avez pas de classe ou de réunion aujourd’hui, vous pouvez étudier, écouter ou lire chez vous. J’ajoute une page à mon site chaque jour, soit une leçon de grammaire ou de vocabulaire, soit un fichier audio, soit une page de compréhension de lecture comme celle-ci. Abonnez-vous à mon infolettre pour recevoir tous ces nouveaux liens deux fois par semaine dans votre boîte à lettres virtuelle, ou bien suivez-moi sur les médias sociaux. Vous pouvez aussi vous familiariser avec les pays et les cultures francophones en écoutant des articles ou en regardant des films, et vous avez la possibilité de suivre l’actualité en lisant un journal français. Peu importe, pourvu que vous passiez du temps avec le français tous les jours.  Autre astuce : suivez-nous sur Facebook, où vous pouvez poser toutes vos questions et discuter des films, de la musique ou encore de la linguistique avec des Francophones et Francophiles du monde entier. C’est un mélange magique d’apprentissage des cultures francophones, des divertissements, de l’amitié – et du français, bien entendu. Et surtout, surtout, n’oubliez pas que vous devez vous engager et faire un effort : trouver de quoi lire, chercher des fichiers audio à écouter, assister aux évènements de votre club, et – je me répète, parce que c’est essentiel – il faut parler, même si vous faites des bêtises. Si vous ne parlez pas, vous perdrez vos connaissances et il deviendra encore plus difficile de parler : c’est un cercle vicieux. En revanche, plus vous parlez, plus vous vous sentirez à l’aise, plus vous aurez envie de parler : un cercle vertueux ! Alors, qu’est-ce que vous attendez ? Au boulot ! Autrefois, les Français faisaient leurs achats dans toute une série de petits magasins spécialisés : le lait et le fromage à la crémerie, la viande chez le boucher, les fruits et les légumes au marché ou bien chez le primeur, et cetera. Depuis des années, avec le développement – et la commodité – des supermarchés, cette habitude se perd. Il est beaucoup plus facile, plus rapide et, en général, moins cher de faire ses courses dans un supermarché, et les petites boutiques disparaissent progressivement.Et ce ne sont pas seulement les magasins d’alimentation qui s’en vont ; les hypermarchés – avec leurs rayons de produits alimentaires, quincaillerie, vêtements, électroménager, jouets et autres produits non comestibles – détournent beaucoup de clients aussi. Résultat : toutes sortes de magasins sont obligés de cesser leurs activités. La seule exception, dans une certaine mesure, semble être les boulangeries. Bref, il se passe en France la même chose que dans beaucoup d’autres pays : les grands magasins mangent les petits, et on échange l’originalité contre la commodité et les prix attractifs. Que l’on soit pour ou contre ces changements, on ne peut pas nier que ça transforme la culture française. Il y a maintes manières de classifier les fromages, et l’une des plus importantes, c’est selon la sorte de lait. Il y en a trois en France : le lait de vache, lait de chèvre et lait de brebis (ce dernier est la femelle du mouton). Une autre classification très importante concerne la méthode de fabrication, à savoir, le type de pâte. Ce sujet est très complexe ; je vous offre juste un aperçu simplifié. Les fromages à pâte fraîche, comme le fromage blanc et la faisselle, ont normalement un goût léger (le chèvre peut être une grande exception). Ces fromages peuvent être fabriqués à partir de lait ou de petit-lait, et se mangent bientôt après leur fabrication. Le fromage à pâte persillée a des moisissures internes et un fort goût : le roquefort, les fromages bleus. Le fromage à pâte pressé ou pâte dure peut être cuit ou pas et est sujet à un affinage entre 3 à 24 mois : beaufort, comté, emmenthal, raclette, mimolette. Signe d’autorité, le premier Grand Sceau de France date du Moyen Âge. Le sceau utilisé de nos jours, défini et frappé en 1848 (IIe République), met en vedette la Liberté, sous forme d’une femme assise, qui tient un faisceau de licteur d’une main et dans l’autre, un gouvernail orné d’un coq gaulois, la patte posée sur un globe. Il y a également une urne, qui rappelle le suffrage universal avec les lettres SU, et des attributs des beaux arts et de l’agriculture à ses pieds. Sur la face est gravé « République française, démocratique, une et indivisible » et au dos « Au nom du peuple français » et « égalité, fraternité ». (La devise française est complète si l’on compte l’image de la Liberté sur l’autre côté.) Ce sceau est repris par les IIIe, IVe et Ve Républiques ; c’est-à-dire que le sceau actuel est le même que celui de 1848. "
new_df = process_text(input_text, 'A2')

new_df = pd.concat([new_df, new_df], ignore_index=True)

input_text = "Dans l’arrière-pays de Provence, à 30 kilomètres au sud-ouest de Draguignan, l’Abbaye du Thoronet est un chef-d’œuvre de l’art roman provençal. Figurant parmi les plus remarquables abbayes cisterciennes, elle est la deuxième des « Trois sœurs provençales ». Construit entre 1160 et 1230, le monastère abritait au début une vingtaine de moines et des dizaines de frères convers (chargés de travaux manuels). D’une conception simple qui profite au maximum de la lumière naturelle, l’abbaye a dû être restaurée à partir de 1841, et l’État a commencé à l’acheter en 1854. Aujourd’hui, elle est ouverte toute l’année à l’exception de 5 jours fériés, et on peut choisir entre une visite commentée ou libre. La direction propose aussi des activités culturelles (conférences, ateliers, concerts), une librairie-boutique et, en été, un resto rapide. Nous avons visité l’Abbaye en novembre, un jour où il faisait gris, et nous n’avons donc malheureusement pas vu la belle luminosité pour laquelle elle est célèbre. Nous avions toujours l’intention de suivre la suggestion de la réceptionniste et y retourner en avril pour une deuxième visite ; hélas, nous ne l’avons jamais fait. Pendant presque tout notre séjour à Hyères, mon mari a donné des cours d’anglais dans une association qui s’appelle l’AVF. L’Accueil des villes françaises est une association non gouvernementale et à but non lucratif dont le personnel se compose de bénévoles. Le but des 350 filiales d’AVF dispersées à travers la France est d’aider les nouveaux arrivants dans une ville à s’y intégrer. Bien qu’affiliées, les associations ne suivent pas toutes le même programme, donc les offres de chacune varient. À Hyères, il y a une trentaine d’ateliers proposés : des langues, des jeux et de l’artisanat. Lors de notre dernière année, mon mari avait animé 4 des 5 niveaux de cours d’anglais, et notre départ a secoué l’organisation. Il a passé des mois à essayer de trouver ses remplaçants, avec peu de succès. En fin de compte, l’AVF a dû annuler plusieurs niveaux – c’était vraiment dommage. S’il n’a pas reçu de salaire, mon mari a été récompensé de plusieurs manières pour son travail à l’AVF. D’abord, il adore enseigner (et il est très doué), donc il éprouve une vraie satisfaction en voyant les progrès de ses étudiants, et en recevant leurs remerciements sincères. Deuxièmement, nous nous sommes fait des dizaines d’amis parmi ses étudiants, dont une dizaine nous ont invités à dîner chez eux (une invitation qu’on prétend être très rare parmi les Français). Et troisièmement, à la fin de l’année, les étudiants dans chaque classe se sont cotisés pour lui acheter des cadeaux de remerciement. Cette année, ils ont été particulièrement généreux, lui offrant des tas de cadeaux (des livres, des DVD, du vin, des cartes-cadeaux), aussi bien qu’un poème et même un sketch taquinant gentiment mon mari pour son style très animé d’enseignement. Comment améliorer son français, comment faire euh moins de fautes, euh comment écrire avec un joli style. Alors, vous entendez souvent les professeurs de français euh vous recommander la lecture et on voit parfois euh des étudiants qui sourient quand on leur fait cette recommendation euh comme pour dire euh « vraiment la lecture euh c’est pas pour moi. » Je voudrais insister là-dessus en mentionnant une étude qui a été faite en 2013 par Alice Sullivan de l’Université de Londres. Cette étude a montré l’immense importance de la lecture. Vous savez certainement qu’il y a plusieurs facteurs qui entre en jeu dans la réussite d’un étudiant. Par exemple, il y a un facteur qui est assez injuste, c’est le niveau de diplôme des parents. On constate, malheureusement, que les étudiants qui ont la chance d’avoir des parents diplômés ont souvent plus de chance de réussite que les autres. Et bien, il faut savoir que le facteur « lecture régulière » est quatre fois plus important dans la réussite que le facteur « niveau de diplôme des parents. » Euh, donc, ce que je conseille vraiment, c’est de se fixer un quart d’heure de lecture par soir. Toujours dans l’objectif d’avoir le courage de s’y mettre, c’est pour un quart d’heure, c’est pas trop long. Évidemment, si, pris dans la lecture on a envie de lire plus, c’est pas grave, c’est très bien. Sinon, si on est fatigué, on se tient vraiment à un quart d’heure, mais un quart d’heure de lecture chaque soir. Alors, ça peut être des livres, peu importe, ce qui vous intéressent, pas trop difficile puisque le but c’est vraiment que vous vous sentiez motivé. Vous allez dans une bibliothèque, une médiathèque et vous dites le type de livre qui vous intéresse a priori, que ça soit roman d’aventure, roman d’amour, roman policier, mais un quart d’heure de lecture par soir. Ça peut même être le journal Équipe, il est bien écrit, mais vous vous en tenez à ce temps de lecture. Autre stratégie pour progresser en français, c’est de vous faire une liste de mots difficiles. Alors, la liste de mots difficiles, eh bien c’est très simple : dès que vous hésitez sur un mot, par exemple, vous avez eu à écrire le mot « développement » et vous avez hésité « il y a un l ? il y a deux p ? » eh bien vous l’écrivez sur votre liste de mots difficiles, et vous relisez régulièrement votre liste de mots difficiles. L’avantage, c’est que cette liste va naturellement compter les mots qui vous viennent le plus facilement sous la plume quand vous écrivez, donc les mots dont vous avez le plus besoin et à force de relire votre liste de mots difficiles, vous allez photographier l’orthographe correcte des mots. Quand on est invité à déjeuner ou à dîner chez des Français, le repas commence normalement avec un apéritif, ou apéro. Avec son choix de boisson et une variété de choses à grignoter, c’est un moment agréable pour bavarder avec les hôtes et les autres invités avant de passer à table. * On peut acheter le vin de noix, vin d’orange et vin de citron, par exemple, dans les magasins de vins, mais beaucoup de nos amis suivent la tradition et font des économies en les fabricant eux-mêmes. Ces vins aux fruits sont, en général, un mélange de vin, alcool de grain, sucre et fruits qu’on a laissé mariner pendant quelques semaines ou mois et qui a une teneur en alcool plus élevée que le vin ordinaire. ** Pas mal de Français semblent croire que les Américains sont fanas de whisky. Une fois, la dame qui nous avait invités à déjeuner chez elle nous a offert, très fièrement, un whisky. Elle nous a raconté l’histoire de son moment de panique en se rendant compte, dans les heures précédant le repas, qu’elle n’en avait pas. Elle a été choquée d’apprendre que mon mari et moi n’aimons pas du tout le whisky. À une autre occasion, lors d’un repas chez nous, l’un des invités nous a offert une bouteille de whisky (que nous avons bien sûr acceptée sans grimacer). Qu’est-ce que la traduction ? C’est la conversion écrite d’une langue, dite la langue « source », en une seconde langue, la langue « cible ». Pour traduire un texte, il faut avoir évidemment une connaissance approfondie des deux langues, mais être bilingue est loin d’être le seul critère. Pour réaliser une bonne traduction, il faut aussi comprendre les cultures rattachées aux langues pour saisir non seulement la signification des mots et expressions, mais aussi l’importance des fêtes, évènements, livres et chansons mentionnés dans le texte. Il est souvent souhaitable d’adapter le texte quand, par exemple, le personnage principal d’un roman déclare que « c’était aussi emballant qu’un match de Coupe du monde ». Pour une audience américaine, on ferait mieux de traduire ce sentiment. Il faut également faire attention aux différences linguistiques. En français, on parle d’une quantité indéfinie importante avec « des dizaines ». Bien que l’on puisse dire tens en anglais, il est beaucoup plus idiomatique de dire dozens, même si le chiffre n’est pas tout à fait le même. Idem pour les expressions idiomatiques. Si la source dit « j’ai un chat dans la gorge », il faut absolument changer l’animal en grenouille pour les Anglophones. Tout aussi important, on doit utiliser des constructions grammaticales convenables à la langue cible. En français, on utilise souvent le futur en décrivant la vie d’un personnage dans une biographie, mais en anglais, le présent ou le conditionnel sont plus appropriés – en apprendre plus. Le principal, c’est non seulement de rester fidèle à la signification des mots, mais de créer un texte dans la langue cible qui se lit comme si c’était l’original. Voilà pourquoi les deux textes ne sont pas identiques : on ne cherche pas une traduction mot-à-mot, mais un texte adapté, voire récrit qui garde et la même signification et le même sentiment que l’original. Quand on habite en France, l’un des meilleurs moyens de rencontrer des gens est de devenir membre d’une association loi de 1901, ce qu’on appellerait en anglais probablement un « club ». Ces associations sont à but non lucratif, et peuvent être déclarées ou pas. Il y a des associations pour tout : pour des sports, des jeux, des activités, des études. Il y a des associations pour les expatriés d’un pays particulier ou pour rassembler tous ceux qui ont autrefois habité dans une certaine région de France. Des associations à but humanitaire luttent pour des individus désavantagés en France ou ailleurs, et des associations créatives réunissent ceux qui aiment danser, chanter ou écrire. Les associations de quartier ou d’arrondissement travaillent pour animer leurs rues, tandis que les associations de tel ou tel festival passent toute l’année à planifier ce seul événement. Il n’est pas toujours facile de trouver une association appropriée. Il y a des annuaires sur internet plus ou moins complets, et on peut toujours demander à l’office de tourisme, mais souvent on découvre une association simplement en parlant avec quelqu’un qui la mentionne. On avait de la chance à Hyères, où on peut obtenir un livret à l’office de tourisme qui répertorie les 300 associations de cette ville de 52 000 habitants. Je recommande l’association Accueil des Villes Françaises pour tout le monde."
new_df = process_text(input_text, 'B1')

new_df = pd.concat([new_df, new_df], ignore_index=True)

input_text = "L’année 1866 fut marquée par un événement bizarre, un phénomène inexpliqué et inexplicable que personne n’a sans doute oublié. Sans parler des rumeurs qui agitaient les populations des ports et surexcitaient l’esprit public à l’intérieur des continents, les gens de mer furent particulièrement émus. Les négociants, armateurs, capitaines de navires, skippers et masters de l’Europe et de l’Amérique, officiers des marines militaires de tous pays, et, après eux, les gouvernements des divers États des deux continents, se préoccupèrent de ce fait au plus haut point. En effet, depuis quelque temps, plusieurs navires s’étaient rencontrés sur mer avec « une chose énorme », un objet long, fusiforme, parfois phosphorescent, infiniment plus vaste et plus rapide qu’une baleine. Les faits relatifs à cette apparition, consignés aux divers livres de bord, s’accordaient assez exactement sur la structure de l’objet ou de l’être en question, la vitesse inouïe de ses mouvements, la puissance surprenante de sa locomotion, la vie particulière dont il semblait doué. Si c’était un cétacé, il surpassait en volume tous ceux que la science avait classés jusqu’alors. Ni Cuvier, ni Lacépède, ni M. Dumeril, ni M. de Quatrefages n’eussent admis l’existence d’un tel monstre—à moins de l’avoir vu, ce qui s’appelle vu de leurs propres yeux de savants. À prendre la moyenne des observations faites à diverses reprises—en rejetant les évaluations timides qui assignaient à cet objet une longueur de deux cents pieds et en repoussant les opinions exagérées qui le disaient large d’un mille et long de trois—on pouvait affirmer, cependant, que cet être phénoménal dépassait de beaucoup toutes les dimensions admises jusqu’à ce jour par les ichtyologistes—s’il existait toutefois. Or, il existait, le fait en lui-même n’était plus niable, et, avec ce penchant qui pousse au merveilleux la cervelle humaine, on comprendra l’émotion produite dans le monde entier par cette surnaturelle apparition. Quant à la rejeter au rang des fables, il fallait y renoncer. Il y avait en Westphalie, dans le château de M. le baron de Thunder-ten-tronckh, un jeune garçon à qui la nature avait donné les mœurs les plus douces. Sa physionomie annonçait son âme. Il avait le jugement assez droit, avec l’esprit le plus simple ; c’est, je crois, pour cette raison qu’on le nommait Candide.* Les anciens domestiques de la maison soupçonnaient qu’il était fils de la sœur de monsieur le baron et d’un bon et honnête gentilhomme du voisinage, que cette demoiselle ne voulut jamais épouser parce qu’il n’avait pu prouver que soixante et onze quartiers, et que le reste de son arbre généalogique avait été perdu par l’injure du temps. Monsieur le baron était un des plus puissants seigneurs de la Westphalie, car son château avait une porte et des fenêtres. Sa grande salle même était ornée d’une tapisserie. Tous les chiens de ses basses-cours composaient une meute dans le besoin ; ses palefreniers étaient ses piqueurs ; le vicaire du village était son grand aumônier. Ils l’appelaient tous monseigneur, et ils riaient quand il faisait des contes. Madame la baronne, qui pesait environ trois cent cinquante livres, s’attirait par là une très grande considération, et faisait les honneurs de la maison avec une dignité qui la rendait encore plus respectable. Sa fille Cunégonde, âgée de dix-sept ans, était haute en couleur, fraîche, grasse, appétissante. Le fils du baron paraissait en tout digne de son père. Le précepteur Pangloss était l’oracle de la maison, et le petit Candide écoutait ses leçons avec toute la bonne foi de son âge et de son caractère. Pangloss enseignait la métaphysico-théologo-cosmolonigologie. Il prouvait admirablement qu’il n’y a point d’effet sans cause, et que, dans ce meilleur des mondes possibles, le château de monseigneur le baron était le plus beau des châteaux et madame la meilleure des baronnes possibles. « Il est démontré, disait-il, que les choses ne peuvent être autrement : car, tout étant fait pour une fin, tout est nécessairement pour la meilleure fin. Remarquez bien que les nez ont été faits pour porter des lunettes, aussi avons-nous des lunettes. Les jambes sont visiblement instituées pour être chaussées, et nous avons des chausses. Les pierres ont été formées pour être taillées, et pour en faire des châteaux, aussi monseigneur a un très beau château ; le plus grand baron de la province doit être le mieux logé ; et, les cochons étant faits pour être mangés, nous mangeons du porc toute l’année : par conséquent, ceux qui ont avancé que tout est bien ont dit une sottise ; il fallait dire que tout est au mieux. » Candide écoutait attentivement, et croyait innocemment; car il trouvait Mlle Cunégonde extrêmement belle, quoiqu’il ne prît jamais la hardiesse de le lui dire. Il concluait qu’après le bonheur d’être né baron de Thunder-ten-tronckh, le second degré de bonheur était d’être Mlle Cunégonde ; le troisième, de la voir tous les jours ; et le quatrième, d’entendre maître Pangloss, le plus grand philosophe de la province, et par conséquent de toute la terre. Un jour, Cunégonde, en se promenant auprès du château, dans le petit bois qu’on appelait parc, vit entre des broussailles le docteur Pangloss qui donnait une leçon de physique expérimentale à la femme de chambre de sa mère, petite brune très jolie et très docile. Comme Mlle Cunégonde avait beaucoup de dispositions pour les sciences, elle observa, sans souffler, les expériences réitérées dont elle fut témoin ; elle vit clairement la raison suffisante du docteur, les effets et les causes, et s’en retourna tout agitée, toute pensive, toute remplie du désir d’être savante, songeant qu’elle pourrait bien être la raison suffisante du jeune Candide, qui pouvait aussi être la sienne. Elle rencontra Candide en revenant au château, et rougit ; Candide rougit aussi ; elle lui dit bonjour d’une voix entrecoupée, et Candide lui parla sans savoir ce qu’il disait. Le lendemain après le dîner, comme on sortait de table, Cunégonde et Candide se trouvèrent derrière un paravent ; Cunégonde laissa tomber son mouchoir, Candide le ramassa, elle lui prit innocemment la main, le jeune homme baisa innocemment la main de la jeune demoiselle avec une vivacité, une sensibilité, une grâce toute particulière ; leurs bouches se rencontrèrent, leurs yeux s’enflammèrent, leurs genoux tremblèrent, leurs mains s’égarèrent. M. le baron de Thunder-ten-tronckh passa auprès du paravent, et voyant cette cause et cet effet, chassa Candide du château à grands coups de pied dans le derrière ; Cunégonde s’évanouit ; elle fut souffletée par Madame la Baronne dès qu’elle fut revenue à elle-même ; et tout fut consterné dans le plus beau et le plus agréable des châteaux possibles. Tout est bien, sortant des mains de l’Auteur des choses ; tout dégénere entre les mains de l’homme. Il force une terre à nourrir les productions d’une autre, un arbre à porter les fruits d’un autre ; il mêle et confond les climats, les éléments, les saisons ; il mutile son chien, son cheval, son esclave ; il bouleverse tout, il défigure tout, il aime la difformité, les monstres ; il ne veut rien tel que l’a fait la nature, pas même l’homme ; il le faut dresser pour lui, comme un cheval de manége ; il le faut contourner à sa mode, comme un arbre de son jardin. Sans cela, tout irait plus mal encore, et notre espèce ne veut pas être façonnée à demi. Dans l’état où sont désormais les choses, un homme abandonné dès sa naissance à lui-même parmi les autres serait le plus défiguré de tous. Les préjugés, l’autorité, la nécessité, l’exemple, toutes les institutions sociales, dans lesquelles nous nous trouvons submergés, étoufferaient en lui la nature, et ne mettraient rien à la place. Elle y serait comme un arbrisseau que le hasard fait naître au milieu d’un chemin, et que les passants font bientôt périr, en le heurtant de toutes parts et le pliant dans tous les sens. C’est à toi que je m’adresse, tendre et prévoyante mère, qui sus t’écarter de la grande route, et garantir l’arbrisseau naissant du choc des opinions humaines ! Cultive, arrose la jeune plante avant qu’elle meure : ses fruits feront un jour tes délices. Forme de bonne heure une enceinte autour de l’âme de ton enfant ; un autre en peut marquer le circuit, mais toi seule y dois poser la barrière. "
new_df = process_text(input_text, 'B2')

new_df = pd.concat([new_df, new_df], ignore_index=True)

input_text = "En 1815, M. Charles-François-Bienvenu Myriel était évêque de Digne. C’était un vieillard d’environ soixante-quinze ans ; il occupait le siège de Digne depuis 1806. Quoique ce détail ne touche en aucune manière au fond même de ce que nous avons à raconter, il n’est peut-être pas inutile, ne fût-ce que pour être exact en tout, d’indiquer ici les bruits et les propos qui avaient couru sur son compte au moment où il était arrivé dans le diocèse. Vrai ou faux, ce qu’on dit des hommes tient souvent autant de place dans leur vie et surtout dans leur destinée que ce qu’ils font. M. Myriel était fils d’un conseiller au parlement d’Aix ; noblesse de robe. On contait de lui que son père, le réservant pour hériter de sa charge, l’avait marié de fort bonne heure, à dix-huit ou vingt ans, suivant un usage assez répandu dans les familles parlementaires. Charles Myriel, nonobstant ce mariage, avait, disait-on, beaucoup fait parler de lui. Il était bien fait de sa personne, quoique d’assez petite taille, élégant, gracieux, spirituel ; toute la première partie de sa vie avait été donnée au monde et aux galanteries. La révolution survint, les événements se précipitèrent, les familles parlementaires décimées, chassées, traquées, se dispersèrent. M. Charles Myriel, dès les premiers jours de la révolution, émigra en Italie. Sa femme y mourut d’une maladie de poitrine dont elle était atteinte depuis longtemps. Ils n’avaient point d’enfants. Que se passa-t-il ensuite dans la destinée de M. Myriel ? L’écroulement de l’ancienne société française, la chute de sa propre famille, les tragiques spectacles de 93, plus effrayants encore peut-être pour les émigrés qui les voyaient de loin avec le grossissement de l’épouvante, firent-ils germer en lui des idées de renoncement et de solitude ? Fut-il, au milieu d’une de ces distractions et de ces affections qui occupaient sa vie, subitement atteint d’un de ces coups mystérieux et terribles qui viennent quelquefois renverser, en le frappant au cœur, l’homme que les catastrophes publiques n’ébranleraient pas en le frappant dans son existence et dans sa fortune ? Nul n’aurait pu le dire ; tout ce qu’on savait, c’est que, lorsqu’il revint d’Italie, il était prêtre. Vers l’époque du couronnement, une petite affaire de sa cure, on ne sait plus trop quoi, l’amena à Paris. Entre autres personnes puissantes, il alla solliciter pour ses paroissiens M. le cardinal Fesch. Un jour que l’empereur était venu faire visite à son oncle, le digne curé, qui attendait dans l’antichambre, se trouva sur le passage de sa majesté. Napoléon, se voyant regardé avec une certaine curiosité par ce vieillard, se retourna, et dit brusquement: L’empereur, le soir même, demanda au cardinal le nom de ce curé, et quelque temps après M. Myriel fut tout surpris d’apprendre qu’il était nommé évêque de Digne. M. Myriel devait subir le sort de tout nouveau venu dans une petite ville où il y a beaucoup de bouches qui parlent et fort peu de têtes qui pensent. Il devait le subir, quoiqu’il fût évêque et parce qu’il était évêque. Mais, après tout, les propos auxquels on mêlait son nom n’étaient peut-être que des propos ; du bruit, des mots, des paroles ; moins que des paroles, des « palabres », comme dit l’énergique langue du midi. Nous étions à l’Étude, quand le Proviseur entra, suivi d’un nouveau habillé en bourgeois et d’un garçon de classe qui portait un grand pupitre. Ceux qui dormaient se réveillèrent, et chacun se leva comme surpris dans son travail. — Monsieur Roger, lui dit-il à demi-voix, voici un élève que je vous recommande, il entre en cinquième.* Si son travail et sa conduite sont méritoires, il passera dans les grands, où l’appelle son âge. Resté dans l’angle, derrière la porte, si bien qu’on l’apercevait à peine, le nouveau était un gars de la campagne, d’une quinzaine d’années environ, et plus haut de taille qu’aucun de nous tous. Il avait les cheveux coupés droit sur le front, comme un chantre de village, l’air raisonnable et fort embarrassé. Quoiqu’il ne fût pas large des épaules, son habit-veste de drap vert à boutons noirs devait le gêner aux entournures et laissait voir, par la fente des parements, des poignets rouges habitués à être nus. Ses jambes, en bas bleus, sortaient d’un pantalon jaunâtre très tiré par les bretelles. Il était chaussé de souliers forts, mal cirés, garnis de clous. Mais, soit qu’il n’eût pas remarqué cette manœuvre ou qu’il n’eut osé s’y soumettre, la prière était finie que le nouveau tenait encore sa casquette sur ses deux genoux. C’était une de ces coiffures d’ordre composite, où l’on retrouve les éléments du bonnet à poil, du chapska, du chapeau rond, de la casquette de loutre et du bonnet de coton, une de ces pauvres choses, enfin, dont la laideur muette a des profondeurs d’expression comme le visage d’un imbécile. Ovoïde et renflée de baleines, elle commençait par trois boudins circulaires ; puis s’alternaient, séparés par une bande rouge, des losanges de velours et de poils de lapin ; venait ensuite une façon de sac qui se terminait par un polygone cartonné, couvert d’une broderie en soutache compliquée, et d’où pendait, au bout d’un long cordon trop mince, un petit croisillon de fils d’or, en manière de gland. Elle était neuve ; la visière brillait. Nous avions l’habitude, en entrant en classe, de jeter nos casquettes par terre, afin d’avoir ensuite nos mains plus libres ; il fallait, dès le seuil de la porte, les lancer sous le banc, de façon à frapper contre la muraille en faisant beaucoup de poussière ; c’était là le genre. On commença la récitation des leçons. Il les écouta de toutes ses oreilles, attentif comme au sermon, n’osant même croiser les cuisses, ni s’appuyer sur le coude, et, à deux heures, quand la cloche sonna, le maître d’études fut obligé de l’avertir, pour qu’il se mît avec nous dans les rangs. Quoi qu’il en fût, après neuf ans d’épiscopat et de résidence à Digne, tous ces racontages, sujets de conversation qui occupent dans le premier moment les petites villes et les petites gens, étaient tombés dans un oubli profond. Personne n’eût osé en parler, personne n’eût même osé s’en souvenir. M. Myriel était arrivé à Digne accompagné d’une vieille fille, mademoiselle Baptistine, qui était sa sœur et qui avait dix ans de moins que lui. Ils avaient pour tout domestique une servante du même âge que mademoiselle Baptistine, et appelée madame Magloire, laquelle, après avoir été « la servante de M. le Curé », prenait maintenant le double titre de femme de chambre de mademoiselle et femme de charge de monseigneur. Mademoiselle Baptistine était une personne longue, pâle, mince, douce ; elle réalisait l’idéal de ce qu’exprime le mot « respectable » ; car il semble qu’il soit nécessaire qu’une femme soit mère pour être vénérable. Elle n’avait jamais été jolie ; toute sa vie, qui n’avait été qu’une suite de saintes œuvres, avait fini par mettre sur elle une sorte de blancheur et de clarté ; et, en vieillissant, elle avait gagné ce qu’on pourrait appeler la beauté de la bonté. Ce qui avait été de la maigreur dans sa jeunesse était devenu, dans sa maturité, de la transparence ; et cette diaphanéité laissait voir l’ange. C’était une âme plus encore que ce n’était une vierge. Sa personne semblait faite d’ombre ; à peine assez de corps pour qu’il y eût là un sexe ; un peu de matière contenant une lueur ; de grands yeux toujours baissés ; un prétexte pour qu’une âme reste sur la terre. Madame Magloire était une petite vieille, blanche, grasse, replète, affairée, toujours haletante, à cause de son activité d’abord, ensuite à cause d’un asthme. À son arrivée, on installa M. Myriel en son palais épiscopal avec les honneurs voulus par les décrets impériaux qui classent l’évêque immédiatement après le maréchal de camp. Le maire et le président lui firent la première visite, et lui de son côté fit la première visite au général et au préfet. L’installation terminée, la ville attendit son évêque à l’œuvre."
new_df = process_text(input_text, 'C1')

new_df = pd.concat([new_df, new_df], ignore_index=True)

input_text = "la liste des produits alternatifs qui visent à « faire aimer » autant qu’à « faire réviser» est longue. Pour éviter d’oublier en deux mois des compétences si chèrement apprises durant l’année, mais aussi pour se cultiver ! Que restera-t-il à la rentrée des acquis de l’année scolaire ? Cerveau et seau de plage jouent-ils les vases communicants Concentrés toute l’année sur les résultats scolaires de leurs enfants, les parents ne sont pas prêts à laisser les bains de mer ou de soleil délaver les tables de multiplication, les plus-que-parfait de l’indicatif et autres théorèmes de géométrie si chèrement acquis. Pour conjurer les deux mois de vacances, ils achètent donc leur potion en librairie pour un investissement moyen de 7 euros. Cela s’appelle cahier de vacances, et 4,5 millions d’écoliers et d’élèves n’ont d’autre solution que de lui trouver une place dans leur valise. Après, c’est une autre affaire. A tel point que, parmi les parents qui ont investi dans ces produits, 4,4 % déclarent que leur enfant ne l’a jamais ouvert… et 72,2 % qu’il ne l’a utilisé qu’en partie. Des chiffres tirés d’une des rares enquêtes sur le sujet, réalisée en 2000 (publiée en 2001) par l’Institut de recherche sur l’économie de l’édu- cation (Irédu) auprès des parents de 2 500 enfants de l’académie de Dijon (Côte-d’Or). Qu’est-ce qui peut bien poser problème dans ces petits livrets pourtant plutôt attractifs pour que seuls 23,4 % de leurs détenteurs arrivent au bout ? Jeune retraité de l’éducation nationale, Roger Rougier a sa réponse. « Je les ai subis, et je les ai fait subir à mes enfants, jusqu’à ce que j’ose m’en affranchir, résume cet inventeur de produits plus ludiques. Je les ai abandonnés le jour où j’ai commencé à faire tenir des cahiers d’été à mes enfants et à créer des jeux avec eux. Je me souviens d’un été durant lequel nous nous étions beaucoup déplacés en caravane. En quittant chaque étape, mes enfants  dessinaient le lieu ou une personne qu’ils y avaient  rencontrée. A la fin des vacances, nous avons assem-  blé ces vignettes et créé ensemble un jeu de l’oie.  Comme support d’échange, ça a été fantastique. Nous  ne pouvions pas faire une partie sans que l’un d’eux  ne raconte un épisode ou un personnage », rappelle  celui qui, ni vu ni connu, a fait, par ce biais,  travailler la narration à ses enfants.  Jouer sur le ludique  Fort de ce succès, Roger Rougier a continué à  développer cette approche amusante des appren-  tissages. Très vite, il s’est mis à créer des fiches de  logique. « Je l’ai fait pour ma classe parce que les  enfants aiment raisonner, que cela les met en  confiance et qu’ensuite ils sont disponibles pour une  leçon », se souvient-il. Ce qui est étonnant, c’est  que les énigmes et les problèmes conçus pour la  classe régalent aussi les petits vacanciers. Expli-  cation de l’auteur : « Ce sont des jeux qui obligent  à une réflexion, un travail intellectuel ; mais l’idée de  jeu masque le côté labeur ». Vendus sous forme de  petits formats noir et blanc, ses Jeux pour s’entraîner    Faut-il enseigner l’informatique à l’école primaire ? Et comment ?  La question n’est plus de savoir s’il faut apprendre l’informatique et son langage, mais de savoir comment,  pour quels usages, et à quelle étape du cursus le faire. […]  DOSSIER  Lisez les documents suivants.  BÂTIR L’ÉCOLE DU XXIe SIÈCLE  Un curseur qu’on a du mal à placer  L’apprentissage du code et plus largement d’une culture générale du numérique à l’école est une mesure récla-  mée depuis longtemps par l’Académie des sciences, soutenue par des acteurs qui comptent dans le secteur  numérique. […] Selon un sondage publié en mai dernier par BVA (société française de sondages) et le Syntec  Numérique (fédération professionnelle du secteur informatique) favorable à la mesure, 87 % des Français  seraient même d’accord pour que la programmation informatique soient enseignée à l’école (24 % à partir du  primaire, 41 % à partir du collège).  Mais cette initiative suscite par ailleurs beaucoup de réticences. On entend souvent l’argument selon lequel  on n’a pas besoin de connaître la mécanique pour apprendre à conduire. Le nombre insuffisant de professeurs  formés est également un frein pour beaucoup d’adversaires de cette mesure, qui la jugent inapplicable.  Entre ceux qui ne jurent que par l’introduction de l’informatique dans l’enseignement obligatoire, et ceux qui  ont peur que l’on veuille transformer le primaire en une grande école d’informatique, il existe pourtant des  pistes pour initier les enseignants et favoriser un passage du périscolaire au scolaire, sans avoir à attendre une  réforme du socle commun* qui prendra des années. L’action de la fondation La Main à la pâte, qui œuvre de-  puis près de 20 ans pour enseigner la science différemment à l’école, est un exemple dont on pourrait s’inspirer.  La Main à la pâte, au service de la science à l’école depuis 20 ans  La Fondation La Main à la pâte a été créée en 2011, dans la continuité de l’opération du même nom lancée en  1995 par l’Académie des sciences à l’initiative du prix Nobel de physique, Georges Charpak. Cette action avait  pour objectif, dès l’origine, d’aider les professeurs à enseigner la science et la technologie en mettant en œuvre  une pédagogie privilégiant l’expérimentation, la discussion, une pratique active et collective. L’idée était de  stimuler chez les élèves l’esprit scientifique et les capacités d’expression, de favoriser leur compréhension du  monde, et de leur permettre de mieux jouer leur rôle de citoyen en proposant des projets pédagogiques orientés  vers des questions de société (éducation à la santé, au développement durable...). Chacun de ces projets touche  en moyenne 10 000 classes. Un beau succès.  Une des originalités de la fondation est d’impliquer la communauté scientifique dans la création des ressources  pédagogiques, l’accompagnement des classes et la formation continue des enseignants. « Ce qui n’a l’air de  rien mais constituait une véritable révolution de palais à l’époque, au ministère de l’éducation nationale »,  confie David Wilgenbus, responsable du secteur production de ressources à La Main à la pâte.  Un enseignement fondé sur l’expérimentation et la pratique, des objectifs d’apprentissage transversaux, des  actions tournées en priorité vers la formation continue des enseignants, menées en collaboration avec la com-  munauté scientifique et professionnelle : le travail de défrichage réalisé par la fondation dans le domaine des  sciences pourrait s’appliquer parfaitement à la problématique de l’enseignement de la culture informatique. décérébrés prenant la place des ouvriers dans les usines. Il vient de la racine slave « robota » qui signifie travail  de force. De là sont nés deux mythes : le robot est un humanoïde et le robot va prendre la place de l’homme.  En parallèle, les romans et les films de science-fiction ont présenté des robots, comme les « Transformers »  par exemple, aux pouvoirs surhumains. La fascination/répulsion des robots vient du respect qu’ils inspirent  pour exécuter des tâches pénibles et nous simplifier la vie, mais qui se mêle à la crainte de les voir prendre le  contrôle de nos vies. J’ai identifié trois peurs fondamentales du robot : la crainte instinctive du métal contre la  chair, la crainte sociétale, presque désespérée du robot qui va voler le travail de l’homme, et, enfin, la crainte  identitaire de voir le robot nous dépasser en capacités intellectuelles ou autres. Il faut juste toujours se rappeler  qu’ils sont avant tout machines savantes et que l’imagination des auteurs est sans limite. Le train, en son temps,  était également craint… […]  Aujourd’hui ou demain, des robots seront-ils capables de remplacer les hommes au travail ? Devien-  dront-ils les esclaves de la société postindustrielle ? Quelles tâches pourraient-ils accomplir ? […]  Bruno Bonnell : Il n’y a aucune tâche que les robots ne sauront pas, un jour, exécuter, de la plus fastidieuse  à la plus méticuleuse, de la plus répétitive à la plus sophistiquée. Qui aurait pensé qu’un jour les enfants de  dix ans pouvaient avoir le monde dans leurs poches au bout d’un portable ? Les véhicules seront autonomes,  plus besoin de chauffeurs, les opérations chirurgicales seront robotisées, les mines seront exploitées par des  robots… Mais utiliser le mot esclave revient à considérer qu’ils sont autre chose que des machines. Leurs  capacités de déduction et de connexion sont certes développées, leur force mécanique est puissante, leur préci-  sion micrométrique, mais les robots restent des machines au service de l’homme. Certains métiers vont dispa-  raître, mais d’autres apparaître. Le maréchal-ferrant et le cocher ont dû se reconvertir. À son essor, l’industrie  de l’automobile a créé beaucoup plus d’emplois qu’elle n’en a supprimés. Il est important d’anticiper cette  « robolution » et de former les gens à des métiers d’avenir pour l’accompagner, sans drame social.  Brigitte Munier : […] L’image d’un robot esclave n’est pas neuve ! Mais on peut aussi imaginer un robot allié  ou compagnon, voire amant. Machine intelligente (capable d’apprentissage), utilisée déjà dans le contexte des  loisirs, de l’éducation, de la médecine, de la police, de l’armée, de la conquête spatiale, etc., le robot est voué  à exécuter toutes les tâches pour lesquelles on voudra, ou pourra, le programmer  "
new_df = process_text(input_text, 'C2')

new_df = pd.concat([new_df, new_df], ignore_index=True)



In [ ]:
print(len(training_data_pd))

### Go Camembert

In [ ]:
import os
from huggingface_hub import login

login(token=os.environ.get("HF_TOKEN"))

In [ ]:
import accelerate
import transformers
import torch
import torch.nn as nn
from transformers import AutoModel
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset, load_metric
import logging
from transformers import DataCollatorWithPadding, DataCollatorForTokenClassification, BitsAndBytesConfig
from accelerate.utils import BnbQuantizationConfig
from accelerate.utils import load_and_quantize_model
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
from sklearn.preprocessing import LabelEncoder

#tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-large',do_lower_case=True)
tokenizer = CamembertTokenizer.from_pretrained('camembert-base',do_lower_case=True)

def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    #labels = df['difficulty'].map({'A1': 1, 'A2': 2, 'B1': 3, 'B2': 4, 'C1': 5, 'C2': 6}).tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    #label_encoder = LabelEncoder()
    #labels = label_encoder.fit_transform(labels)
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

train_data, val_data = train_test_split(training_data_pd, test_size=0.1, random_state=42)

#train_data = pd.concat([train_data, new_df], ignore_index=True)

'''
#shuffle
train_data = pd.concat([train_data, train_data], ignore_index=True)
train_data = train_data.sample(frac=1, random_state=42).reset_index(drop=True)
'''

train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)

#model = CamembertForSequenceClassification.from_pretrained('camembert/camembert-large', num_labels=6)
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

In [ ]:
print(train_dataset['labels'])

In [ ]:
#model = model.to('cuda')

import json
import wandb

run = wandb.init(
    project='Cam01', 
    name="031", 
)

def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

training_args = TrainingArguments(
    gradient_accumulation_steps=1,
    output_dir='./results',
    num_train_epochs=6,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=50,
    learning_rate=5e-5,
    #adam_epsilon=10e-3,
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

args_dict = training_args.to_dict()

# Save the dictionary to a JSON file
with open('training_arguments031.json', 'w') as f:
    json.dump(args_dict, f, indent=4)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #tokenizer='camembert-base',
    #data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]

)

trainer.train()

In [ ]:
#don't do before the eval
wandb.finish()

In [ ]:
eval_results = trainer.evaluate()
print("Validation Results:", eval_results)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

predictions = trainer.predict(val_dataset)

preds = np.argmax(predictions.predictions, axis=-1)

#print(val_dataset['labels'])

difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in preds]
original_difficulties = [difficulty_levels[label] for label in val_dataset['labels']]

cm = confusion_matrix(original_difficulties, predicted_difficulties, labels=["A1","A2","B1","B2","C1","C2"])
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=["A1","A2","B1","B2","C1","C2"], yticklabels=["A1","A2","B1","B2","C1","C2"])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

#### for unlabelled

In [ ]:
from transformers import CamembertTokenizer
import numpy as np

#------------ must change model
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

max_length = 512 
unlabelled_encodings = tokenizer(list(unlabelled_data_pd['sentence']), truncation=True, padding=True, max_length=max_length, return_tensors="pt")

from torch.utils.data import Dataset, DataLoader

class SimpleDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

dataset = SimpleDataset(unlabelled_encodings)

predictions = trainer.predict(dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

difficulty_levels = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}
predicted_difficulties = [difficulty_levels[label] for label in predicted_labels]

import pandas as pd

submission_df = pd.DataFrame({
    'id': unlabelled_data_pd['id'],
    'difficulty': predicted_difficulties
})

submission_df.to_csv('F:\projects\Matthias\proj_01\cam01\submission_CamemBERT_018.csv', index=False)

